#### Wesleyan University ASTR 221

## Tutorial 5: Numerical Differentiation (Finite Differences)

There are many circumstances when one might have some discrete data points and want to know the *derivative* of it, i.e., how it is changing.  (For example, when estimating some other quantity that depends on the spatial or temporal gradient of your data, such as the force exerted on a fluid when there is a spatial gradient in pressure.)  The most basic methods for estimating the derivative numerically are called **finite-difference methods**.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120  
from math import pi

Let's return to our sine wave, sampled every $\pi/3$ radians:

In [ ]:
# Construct array of equally-spaced grid points
xmin = 0.
xmax = 2*pi
npts = 7
dx = (xmax - xmin)/(npts-1)

# Sample points
x = np.linspace(xmin, xmax, npts)
y = np.sin(x)

# Make a very high-resolution x-grid to show the underlying sine curve
x_hr = np.linspace(xmin, xmax, 200)
y_hr = np.sin(x_hr)

fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', lw=2, color='deepskyblue')
plt.plot(x, y, 'o', ms=5, color='red')
ax.set_xlabel('x')
ax.set_ylabel('y')

Because we know that the underlying function is a sine curve, we know its exact derivative: $f'(x) = \cos(x)$.  However, in most cases you do *not* know the underlying function, and have to estimate its derivative from the sampled points.  

The most straightforward way to estimate the derivative is to measure the slope of the line between neighboring points.  To estimate the derivative at a given sample point $x_i$, one can use the slope between $x_i$ and $x_{i+1}$ (*forward-difference*), $x_i$ and $x_{i-1}$ (*backward-difference*), or $x_{i-1}$ and $x_{i+1}$ (*central-difference*).  The expressions for these are:

**Forward-difference:** 
$$f'(x_i) \approx \frac{f(x_{i+1})-f(x_i)}{x_{i+1} - x_i}$$

**Backward-difference:** 
$$f'(x_i) \approx \frac{f(x_{i})-f(x_{i-1})}{x_{i} - x_{i-1}}$$

**Central-difference:** 
$$f'(x_i) \approx \frac{f(x_{i+1})-f(x_{i-1})}{x_{i+1} - x_{i-1}}$$

Below I have written functions that calculate the derivative at sample point $x_i$ with each of these three methods:

In [ ]:
# Given a set of points (xp, yp), calculates the derivative at xp[i]
# using the forward difference method
def forward_diff_single(i, xp, yp):
    if i >= len(xp)-1 or i < 0:
        print("Index outside the range allowed for forward difference")
        return
    
    return (yp[i+1] - yp[i])/(xp[i+1] - xp[i])

# Given a set of points (xp, yp), calculates the derivative at xp[i]
# using the backward difference method
def backward_diff_single(i, xp, yp):
    if i <= 0 or i > len(xp)-1:
        print("Index outside the range allowed for backward difference")
        return
    
    return (yp[i] - yp[i-1])/(xp[i] - xp[i-1])

# Given a set of points (xp, yp), calculates the derivative at xp[i]
# using the central difference method
def central_diff_single(i, xp, yp):
    if i <= 0 or i >= len(xp)-1:
        print("Index outside the range allowed for central difference")
        return
    
    return (yp[i+1] - yp[i-1])/(xp[i+1] - xp[i-1])

Let's see how each of these functions performs for the third point on our sine curve.  On top of the intrinsic sine curve and the sample points, below I plot three lines in point-slope form,
$$ y = m*(x-x_i) + y_i $$
where $m$ is the slope returned by each of the functions above and $(x_i, y_i)$ is the sample point.

In [ ]:
fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', color='deepskyblue', lw=2)
plt.plot(x, y, 'o', ms=5, color='red')
ax.set_xlabel('x')
ax.set_ylabel('y')

i = 2

plt.plot(x_hr, forward_diff_single(i, x, y)*(x_hr - x[i]) + y[i], '-', color='gold', label='forward')
plt.plot(x_hr, backward_diff_single(i, x, y)*(x_hr - x[i]) + y[i], '-', color='violet', label='backward')
plt.plot(x_hr, central_diff_single(i, x, y)*(x_hr - x[i]) + y[i], '-', color='green', label='central')

ax.set_ylim(-1.3, 1.3)
ax.legend()

Note that the forward difference line goes through the sample point and the next point, the backward difference line goes through the sample point and the previous point, and the central difference line goes through only the sample point but is parallel to the line connecting the next and previous points.

Now let's write a more general set of functions that will calculate the derivative over the whole array of sample points.  The functions below are equivalent to the ones above, except that they make the calculation over the whole array rather than a single point at a time.  To understand what is going on with the array notation below, recall that

```x[1:] = [x[1], x[2], x[3], ... , x[n-1]]```

and

```x[:-1] = [x[0], x[1], x[2], ... , x[n-2]]```

such that the difference of the two arrays is

```x[1:] - x[:-1] = [x[1] - x[0], x[2] - x[1], x[3] - x[2], ... , x[n-1] - x[n-2]]```

Other than the array notation, the main difference between the functions below and the single-point functions above is that none of the finite-difference methods can be applied over the entire array: they are all inapplicable at at least one endpoint.  So, there is also a line in each function to apply a compatible method to the missing endpoint(s).

In [ ]:
# Returns an array of forward differences corresponding to each point in the given arrays
# Uses backward difference at the last point
def forward_diff_full(xp, yp):
    yprime = 0.*yp
    yprime[:-1] = (yp[1:] - yp[:-1])/(xp[1:] - xp[:-1])
    yprime[-1] = backward_diff_single(len(xp)-1, xp, yp)
    
    return yprime

# Returns an array of backward differences corresponding to each point in the given arrays
# Uses forward difference at the first point
# Note how this is identical to forward_diff_full, except it assigns the derivative to the other endpoint!
def backward_diff_full(xp, yp):
    yprime = 0.*yp
    yprime[1:] = (yp[1:] - yp[:-1])/(xp[1:] - xp[:-1])
    yprime[0] = forward_diff_single(0, xp, yp)
    
    return yprime

# Returns an array of central differences corresponding to each point in the given arrays
# Uses backward difference at the last point and forward difference at the first point
def central_diff_full(xp, yp):
    yprime = 0.*yp
    yprime[1:-1] = (yp[2:] - yp[:-2])/(xp[2:] - xp[:-2])
    yprime[0] = forward_diff_single(0, xp, yp)
    yprime[-1] = backward_diff_single(len(xp)-1, xp, yp)
    
    return yprime

Let's see how these finite-difference estimates stack up against the exact solution, $f'(x) = \cos(x)$.  The block below calculates the derivatives and plots them directly:

In [ ]:
yprime_hr = np.cos(x_hr)

fig, ax = plt.subplots()
plt.plot(x_hr, yprime_hr, '-', color='deepskyblue', lw=3, label='exact')
plt.plot(x, forward_diff_full(x, y), 'o-', ms=5, color='gold', label='forward')
plt.plot(x, backward_diff_full(x, y), 'o-', ms=5, color='violet', label='backward')
plt.plot(x, central_diff_full(x, y), 'o-', ms=5, color='green', label='central')
ax.set_xlabel('x')
ax.set_ylabel('y\'')
ax.legend()

Not bad!  You can already see by eye that the central-difference method is more accurate than the forward- or backward-difference methods.  All the methods will get more accurate as the distance between points $\Delta x$ decreases.  Try messing around with the value of ```npts``` in the block below, which repeats the calculation from above with the given number of sample points.

In addition, we plot the result from numpy's ```gradient()``` function - which, as you will see, is identical to our central-difference approximation!

In [ ]:
npts = 10
dx = (xmax - xmin)/(npts-1)

# Sample points
x = np.linspace(xmin, xmax, npts)
y = np.sin(x)

yprime_hr = np.cos(x_hr)

fig, ax = plt.subplots()
plt.plot(x_hr, yprime_hr, '-', color='deepskyblue', lw=3, label='exact')
plt.plot(x, forward_diff_full(x, y), 'o-', ms=5, color='gold', label='forward')
plt.plot(x, backward_diff_full(x, y), 'o-', ms=5, color='violet', label='backward')
plt.plot(x, central_diff_full(x, y), 'o-', ms=5, color='green', label='central')
plt.plot(x, np.gradient(y, x[1]-x[0]), 'o', ms=2, color='lawngreen', label='numpy')
ax.set_xlabel('x')
ax.set_ylabel('y\'')
ax.legend()

Finally, here's a more complicated function to play around with.  The code below repeats the exercise from above, but with the complex function we used in Tutorial 4 to test our interpolation schemes.  Its derivative is even more complicated, but our finite-difference methods do a pretty good job of capturing it for npts > 15 or so.

In [ ]:
npts = 15
dx = (xmax - xmin)/(npts-1)

# Sample points
x = np.linspace(xmin, xmax, npts)
y = np.exp(-x/5)*(np.sin(x/1.5) - np.cos(x*3)/2)

y_hr = np.exp(-x_hr/5)*(np.sin(x_hr/1.5) - np.cos(x_hr*3)/2)
yprime_hr = 0.1*np.exp(-x_hr/5)*(-2*np.sin(2*x_hr/3) + 15*np.sin(3*x_hr) + 20./3*np.cos(2*x_hr/3) + np.cos(3.*x_hr))

fig, ax = plt.subplots()
plt.plot(x_hr, y_hr, '-', color='deepskyblue', lw=3, label='exact')
plt.plot(x, y, 'o', color='red', ms=5, label='samples')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend()

fig, ax = plt.subplots()
plt.plot(x_hr, yprime_hr, '-', color='deepskyblue', lw=3, label='exact')
plt.plot(x, forward_diff_full(x, y), 'o-', ms=5, color='gold', label='forward')
plt.plot(x, backward_diff_full(x, y), 'o-', ms=5, color='violet', label='backward')
plt.plot(x, central_diff_full(x, y), 'o-', ms=5, color='green', label='central')
ax.set_xlabel('x')
ax.set_ylabel('y\'')
ax.legend()